### <br> Importacion de librerias <br/>

In [1]:
import pyodbc

In [2]:
# Configura la cadena de conexión para tu base de datos
conn_str = (
    r'DRIVER={SQL Server};'
    r'SERVER=DESKTOP-5U13SVB\SQLEXPRESS;'  # Reemplaza con el nombre de tu servidor
    r'DATABASE=AviacionComercial;'
    r'Trusted_Connection=yes;'
)

# Conecta a la base de datos
connection = pyodbc.connect(conn_str)

# Ahora puedes utilizar la conexión para ejecutar consultas


In [4]:
def create_tables(connection):
    cursor = connection.cursor()

    # Consulta SQL para crear las tablas
    create_table_queries = [
        '''
        CREATE TABLE aircrafts_data (
            aircraft_code VARCHAR(10) PRIMARY KEY NOT NULL,
            range INT NOT NULL,
            model_en VARCHAR(30) NOT NULL
        );
        ''',
        '''
        CREATE TABLE airports_data (
            airport_code VARCHAR(10) PRIMARY KEY NOT NULL,
            airport_name VARCHAR(100) NOT NULL,
            city VARCHAR(30) NOT NULL,
            timezone VARCHAR(30) NOT NULL,
            longitude VARCHAR(100) NOT NULL,
            latitude VARCHAR(100) NOT NULL
        );
        ''',
        '''
        CREATE TABLE bookings (
            book_ref VARCHAR(15) PRIMARY KEY NOT NULL,
            book_date DATETIME,
            total_amount DECIMAL(15,2)
        );
        ''',
        '''
        CREATE TABLE flights (
            flight_id INT PRIMARY KEY NOT NULL,
            flight_no VARCHAR(10),
            scheduled_departure DATETIME,
            scheduled_arrival DATETIME,
            departure_airport VARCHAR(10),
            arrival_airport VARCHAR(10),
            status VARCHAR(30),
            aircraft_code VARCHAR(10),
            actual_departure DATETIME,
            actual_arrival DATETIME,
            FOREIGN KEY (departure_airport) REFERENCES airports_data(airport_code),
            FOREIGN KEY (arrival_airport) REFERENCES airports_data(airport_code),
            FOREIGN KEY (aircraft_code) REFERENCES aircrafts_data(aircraft_code)
        );
        ''',
        '''
        CREATE TABLE seats (
            seat_id VARCHAR(100) PRIMARY KEY NOT NULL,
            aircraft_code VARCHAR(10),
            seat_no VARCHAR(10),
            fare_conditions VARCHAR(30),
            FOREIGN KEY (aircraft_code) REFERENCES aircrafts_data(aircraft_code)
        );
        ''',
        '''
        CREATE TABLE tickets (
            ticket_no VARCHAR(100) PRIMARY KEY NOT NULL,
            passenger_id VARCHAR(20),
            book_ref VARCHAR(15),
            FOREIGN KEY (book_ref) REFERENCES bookings(book_ref)
        );
        ''',
        '''
        CREATE TABLE boarding_passes (
            bp_id VARCHAR(100) PRIMARY KEY NOT NULL,
            ticket_no VARCHAR(100),
            flight_id INT,
            boarding_no INT,
            seat_no VARCHAR(10),
            FOREIGN KEY (ticket_no) REFERENCES tickets(ticket_no)
        );
        ''',
        '''
        CREATE TABLE ticket_flights (
            ticket_flights_id VARCHAR(100) PRIMARY KEY NOT NULL,
            ticket_no VARCHAR(100),
            flight_id INT,
            fare_conditions VARCHAR(30),
            amount DECIMAL(15,2),
            FOREIGN KEY (ticket_no) REFERENCES tickets(ticket_no),
            FOREIGN KEY (flight_id) REFERENCES flights(flight_id)
        );
        '''
    ]

    # Ejecutar cada consulta para crear las tablas
    for query in create_table_queries:
        try:
            cursor.execute(query)
            connection.commit()
            print("Tabla creada exitosamente.")
        except Exception as e:
            print(f'Error al crear tabla: {e}')
            connection.rollback()

    cursor.close()

# Configura la cadena de conexión para tu base de datos
conn_str = (
    r'DRIVER={SQL Server};'
    r'SERVER=DESKTOP-5U13SVB\SQLEXPRESS;'  
    r'DATABASE=AviacionComercial;'
    r'Trusted_Connection=yes;'
)

# Conecta a la base de datos y crea las tablas
connection = pyodbc.connect(conn_str)
create_tables(connection)
connection.close()



Tabla creada exitosamente.
Tabla creada exitosamente.
Tabla creada exitosamente.
Tabla creada exitosamente.
Tabla creada exitosamente.
Tabla creada exitosamente.
Tabla creada exitosamente.
Tabla creada exitosamente.


In [5]:
import pandas as pd
import os
import logging

In [6]:
# Configuración del archivo de log
log_file = r"C:\users\usuario\Desktop\FFFF\proyecto_final_henry\conn_autom_bd\ingesta.txt"

def log_update(message):
    with open(log_file, 'a') as f:
        f.write(message + '\n')

def load_data_to_sql(file_path, table_name, connection):
    if not os.path.isfile(file_path):
        log_update(f'Error: El archivo {file_path} no existe.')
        return 0
    
    try:
        # Lee el archivo CSV
        df = pd.read_csv(file_path)
        
        # Conexión a la base de datos
        cursor = connection.cursor()
        
        # Inserta los datos en la tabla
        for _, row in df.iterrows():
            columns = ', '.join(df.columns)
            placeholders = ', '.join('?' * len(row))
            insert_sql = f"INSERT INTO {table_name} ({columns}) VALUES ({placeholders})"
            cursor.execute(insert_sql, tuple(row))
        
        # Confirma los cambios
        connection.commit()
        new_records_count = len(df)
        log_update(f'Cantidad de registros nuevos añadidos a {table_name}: {new_records_count}.')
    
    except Exception as e:
        log_update(f'Error al insertar datos en {table_name}: {str(e)}')
    
    finally:
        cursor.close()

def main():
    server = 'DESKTOP-5U13SVB\SQLEXPRESS;'  
    data_folder = r'C:\users\usuario\Desktop\FFFF\proyecto_final_henry\data'
    files_and_tables = {
        'aircrafts_data.csv': 'aircrafts_data',
        'airports_data.csv': 'airports_data',
        'bookings.csv': 'bookings',
        'flights.csv': 'flights',
        'seats.csv': 'seats',
        'tickets.csv': 'tickets',
        'boarding_passes.csv': 'boarding_passes',
        'ticket_flights.csv': 'ticket_flights'
    }

    conn_str = f'DRIVER={{SQL Server}};SERVER={server};DATABASE=AviacionComercial;Trusted_Connection=yes;'
    connection = pyodbc.connect(conn_str)
    
    try:
        for file_name, table_name in files_and_tables.items():
            file_path = os.path.join(data_folder, file_name)
            load_data_to_sql(file_path, table_name, connection)
    
    finally:
        connection.close()

if __name__ == "__main__":
    main()



<>:39: SyntaxWarning: invalid escape sequence '\S'
<>:39: SyntaxWarning: invalid escape sequence '\S'
C:\Users\Usuario\AppData\Local\Temp\ipykernel_19684\673705649.py:39: SyntaxWarning: invalid escape sequence '\S'
  server = 'DESKTOP-5U13SVB\SQLEXPRESS;'


<br>Automatizacion<br/>

### Opcion 1

In [9]:
!pip install schedule
import schedule

Defaulting to user installation because normal site-packages is not writeable


In [11]:
import time 

In [12]:
def job():
    print("Iniciando el proceso de carga de datos...")
    main()

# Programa el proceso para que se ejecute cada día a las 12:00 PM
schedule.every().day.at("12:00").do(job)

while True:
    schedule.run_pending()
    time.sleep(60)  # Revisa cada minuto si es el momento de ejecutar el proceso


### Opcion 2

In [ ]:
import time

def automate_data_loading(interval_minutes=60):
    while True:
        print("Iniciando el proceso de carga de datos...")
        main()  # Ejecuta el proceso principal de carga de datos
        print(f"Proceso completado. Esperando {interval_minutes} minutos para la siguiente ejecución...")
        time.sleep(interval_minutes * 60)  # Pausa antes de la siguiente ejecución

# Puedes ajustar el intervalo según sea necesario (por ejemplo, cada 60 minutos)
automate_data_loading(interval_minutes=60)
